# Rowing Classfier

## This projects aims to classify images of rowing boats into their respective boat classes.

Let's import the modules we need - we'll use the ResNet50 model as the base of our model. It's been trained on the ImageNet dataset to detect and classify 1,000 objects. 

In [1]:
import keras
from keras.applications.resnet50 import ResNet50
from keras.preprocessing import image
from keras.applications.resnet50 import preprocess_input, decode_predictions
from keras.preprocessing.image import ImageDataGenerator

from keras.models import Sequential
from keras.layers import Flatten, Dense, Dropout

Using TensorFlow backend.


Now let's define the model and compile it.

In [2]:
num_classes = 2
#image size is 600x400
#but we can resize it to 224x224

model = Sequential()
model.add(ResNet50(include_top=False, weights='imagenet',  pooling='avg', input_shape=(224,224,3)))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.25))
model.add(Dense(32, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

model.layers[0].trainable=False

W0817 20:42:01.374658 19892 deprecation_wrapper.py:119] From C:\Users\md705\Anaconda3\envs\rowing\lib\site-packages\keras\backend\tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0817 20:42:01.399693 19892 deprecation_wrapper.py:119] From C:\Users\md705\Anaconda3\envs\rowing\lib\site-packages\keras\backend\tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0817 20:42:01.405714 19892 deprecation_wrapper.py:119] From C:\Users\md705\Anaconda3\envs\rowing\lib\site-packages\keras\backend\tensorflow_backend.py:4185: The name tf.truncated_normal is deprecated. Please use tf.random.truncated_normal instead.

W0817 20:42:01.442199 19892 deprecation_wrapper.py:119] From C:\Users\md705\Anaconda3\envs\rowing\lib\site-packages\keras\backend\tensorflow_backend.py:174: The name tf.get_default_session is deprecated. Please use tf.compat.v1.get_default_session instead

A local file was found, but it seems to be incomplete or outdated because the md5 file hash does not match the original value of a268eb855778b3df3c7506639542a6af so we will re-download the data.
94658560/94653016 [==============================] - 308s 3us/step


W0817 20:47:32.880797 19892 deprecation.py:506] From C:\Users\md705\Anaconda3\envs\rowing\lib\site-packages\keras\backend\tensorflow_backend.py:3445: calling dropout (from tensorflow.python.ops.nn_ops) with keep_prob is deprecated and will be removed in a future version.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [3]:
model.compile(optimizer='adam',
             loss='categorical_crossentropy',
             metrics=['accuracy'])

W0817 20:48:51.635978 19892 deprecation_wrapper.py:119] From C:\Users\md705\Anaconda3\envs\rowing\lib\site-packages\keras\optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.



In [4]:
#for training dataset
aug_datagen = ImageDataGenerator(rotation_range = 30,
                                    width_shift_range = 0.2,
                                    height_shift_range = 0.2,
                                    zoom_range = 0.1,
                                    horizontal_flip = True,
                                    rescale = 1./255,
                                    validation_split = 0.2
                                    )

#for testing data set
test_datagen = ImageDataGenerator(rescale = 1./255)

In [10]:
train_generator = aug_datagen.flow_from_directory('images/train',
                                                 target_size=(224, 224),
                                                 batch_size=8,
                                                 class_mode='categorical',
                                                 subset='training')

val_generator = aug_datagen.flow_from_directory('images/train',
                                                 target_size=(224, 224),
                                                 class_mode='categorical',
                                                 subset='validation')

test_generator = test_datagen.flow_from_directory('images/test',
                                                 target_size=(224,224),
                                                 class_mode='categorical')

Found 16 images belonging to 2 classes.
Found 4 images belonging to 2 classes.
Found 10 images belonging to 2 classes.


In [12]:
model.fit_generator(train_generator,
                   steps_per_epoch=2,
                   epochs = 2,
                   validation_data=val_generator,
                   validation_steps=1)

W0817 20:54:07.624962 19892 deprecation.py:323] From C:\Users\md705\Anaconda3\envs\rowing\lib\site-packages\tensorflow\python\ops\math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Epoch 1/2
2/2 [==============================] - 12s 6s/step - loss: 1.0423 - acc: 0.3750 - val_loss: 1.0269 - val_acc: 0.5000
Epoch 2/2
2/2 [==============================] - 6s 3s/step - loss: 0.5848 - acc: 0.8125 - val_loss: 0.9922 - val_acc: 0.5000
